# 02. Treinamento de Modelos

Este notebook demonstra como utilizar o `TrainingService` para treinar modelos de detecção de deepfake utilizando os datasets processados.

In [ ]:
# 1. Configuração do Ambiente
import os
import sys
import numpy as np

def is_colab():
    return 'google.colab' in sys.modules

if is_colab():
    if not os.path.exists('/content/TCC'):
        !git clone https://github.com/thieryw/TCC.git /content/TCC
    os.chdir('/content/TCC')
    !pip install -r requirements.txt
    sys.path.append('/content/TCC')
    
    # Montar Drive para salvar modelos
    from app.utils.colab import mount_drive
    mount_drive()
else:
    if os.path.exists('app'):
        sys.path.append(os.getcwd())

In [ ]:
# 2. Preparação dos Dados (Exemplo)
# Vamos criar um dataset sintético para demonstração se não houver um real.

DATASET_PATH = "datasets/processed/dataset_example.npz"
os.makedirs(os.path.dirname(DATASET_PATH), exist_ok=True)

if not os.path.exists(DATASET_PATH):
    print("Criando dataset sintético para teste...")
    # 100 amostras, input shape (128, 128) - ex: espectrogramas
    X_train = np.random.rand(100, 128, 128).astype(np.float32)
    y_train = np.random.randint(0, 2, 100).astype(np.float32) # 0 ou 1
    
    X_val = np.random.rand(20, 128, 128).astype(np.float32)
    y_val = np.random.randint(0, 2, 20).astype(np.float32)
    
    np.savez(DATASET_PATH, X_train=X_train, y_train=y_train, X_val=X_val, y_val=y_val)
    print(f"Dataset salvo em: {DATASET_PATH}")
else:
    print(f"Usando dataset existente: {DATASET_PATH}")
    
data = np.load(DATASET_PATH)
print("Shape X_train:", data['X_train'].shape)
print("Shape y_train:", data['y_train'].shape)

## 3. Configuração e Treinamento
Utilizaremos o `TrainingService` para treinar uma arquitetura disponível (ex: `mesonet` ou `simple_cnn`).

In [ ]:
from app.domain.services.training_service import TrainingService

# Inicializar serviço
# models_dir define onde os modelos treinados serão salvos
MODELS_DIR = "app/models" 
if is_colab() and os.path.exists('/content/drive/MyDrive'):
    MODELS_DIR = "/content/drive/MyDrive/TCC_Models"

training_service = TrainingService(models_dir=MODELS_DIR)

# Configuração do treinamento
config = {
    "parameters": {
        "epochs": 5,
        "batch_size": 32,
        "learning_rate": 0.001
    }
}

# Iniciar treinamento
# Certifique-se que a arquitetura 'simple_cnn' ou similar existe no registro
architecture_name = "mesonet" # ou outro registrado

print(f"Iniciando treinamento com arquitetura: {architecture_name}...")
result = training_service.train_model(
    architecture=architecture_name,
    dataset_path=DATASET_PATH,
    config=config
)

if result.status.name == 'SUCCESS':
    print("Treinamento concluído!")
    print("Metadados do Modelo:", result.data)
else:
    print("Erro no treinamento:", result.errors)

## 4. Análise dos Resultados
Visualizar métricas de treinamento.

In [ ]:
if result.status.name == 'SUCCESS':
    metrics = result.data.metrics
    print("Métricas finais:", metrics)
    
    # Se houver histórico de loss/accuracy salvo nos metadados ou retornado
    # (Depende da implementação do ModelTrainer retornar history)
    # Aqui plotaríamos gráficos se disponíveis.